# Loading data / basic formatting

### == receipts data ==

In [1]:
import pandas as pd
import numpy as np

In [2]:
# basic loading: nulls and datetime format
r = pd.read_csv('data/receipts.csv', na_values='(null)', parse_dates=['created_at'])
r

,id,customer_id,retailer_id,created_at,total_price
0,105181352,23001,133.0,2016-09-22 01:17:36,NaN
1,119833247,23001,133.0,2016-11-22 13:40:21,NaN
2,107540410,93001,49.0,2016-10-01 21:16:59,34.20
3,118306190,119001,7.0,2016-11-16 19:49:04,78.27
4,105456304,177001,18.0,2016-09-23 05:34:40,43.66
...,...,...,...,...,...
22316,122777856,11025001,49.0,2016-12-03 20:59:30,8.55
22317,123487988,13097001,188.0,2016-12-06 01:40:33,23.91
22318,123471468,13374001,4.0,2016-12-06 00:51:29,147.00
22319,122634307,13848001,12.0,2016-12-03 12:45:46,20.79


In [3]:
# ~9% have no price info (different from 0)
r.isna().sum()

# r[r['total_price'] == 0]

id                0
customer_id       0
retailer_id       1
created_at        0
total_price    2342
dtype: int64

In [4]:
# One odd row with missing retailer.
r[r['retailer_id'].isna()]

,id,customer_id,retailer_id,created_at,total_price
21405,109936097,1,NaN,2016-10-11 22:20:08,NaN


In [5]:
# Customer id is 1 meaning this is likely a test of some kind. Safe to drop.
r.dropna(subset=['retailer_id'], inplace=True)

### == receipt items data ==

In [6]:
# basic loading: nulls
ri = pd.read_csv('data/receipt_items.csv', na_values='(null)')
ri

,receipt_item_id,price,quantity
0,1931607223,4.97,NaN
1,1931607226,NaN,NaN
2,1931875497,NaN,1.0
3,1931788508,NaN,NaN
4,1931788509,NaN,NaN
...,...,...,...
240548,2320623129,NaN,NaN
240549,2320623132,NaN,NaN
240550,2320623135,NaN,NaN
240551,2320623139,NaN,NaN


In [7]:
# most rows missing one or both columns
ri.isna().sum()

receipt_item_id         0
price              143722
quantity           157633
dtype: int64

In [8]:
# only ~5% have both
ri[~ri.isna().any(axis=1)]

,receipt_item_id,price,quantity
91,1973681370,0.88,1.0
92,1973681372,2.99,1.0
93,1973681375,0.99,1.0
94,1973681378,1.99,1.0
108,1980434188,1.18,1.0
...,...,...,...
240028,2266397705,28.64,1.0
240029,2266397714,41.44,1.0
240212,2280002756,0.78,2.0
240213,2280002762,0.05,1.0


### == receipt item details ==

In [9]:
rid = pd.read_csv('data/receipt_item_details.csv', na_values='(null)')
rid

,receipt_item_id,receipt_id,primary_category_id,secondary_category_id,tertiary_category_id,brand_id,global_product_id
0,1930961526,101782310,130.0,131.0,131.0,4477.0,525012
1,1930961527,101782310,NaN,NaN,NaN,34236.0,891808
2,1930961529,101782310,237.0,240.0,246.0,34246.0,156367
3,1930961531,101782310,130.0,132.0,132.0,4517.0,71186
4,1930961532,101782310,223.0,369.0,369.0,17407.0,750345
...,...,...,...,...,...,...,...
240546,2318337713,123401030,237.0,255.0,262.0,34246.0,751626
240547,2319598027,123464264,58.0,70.0,70.0,677.0,5973
240548,2319598033,123464264,58.0,70.0,70.0,708.0,5991
240549,2319598053,123464264,22.0,23.0,23.0,1019.0,9343


In [10]:
# ~9% missing categories, some also missing brands
rid.isna().sum()

receipt_item_id              0
receipt_id                   0
primary_category_id      21736
secondary_category_id    21736
tertiary_category_id     21736
brand_id                  4151
global_product_id            0
dtype: int64

# Analysis

### == duplicate row analysis ==

In [11]:
# ri and rid have different numbers of rows?

ri['receipt_item_id'].value_counts().head()

1992189181    4
1931607223    1
2071052834    1
2070871352    1
2070871353    1
Name: receipt_item_id, dtype: int64

In [12]:
rid['receipt_item_id'].value_counts().head()

1992189181    2
1930961526    1
2296648820    1
2296648726    1
2296648731    1
Name: receipt_item_id, dtype: int64

In [13]:
# This receipt item is repeated on two separate receipts for whatever reason
rid[rid['receipt_item_id'] == 1992189181]

,receipt_item_id,receipt_id,primary_category_id,secondary_category_id,tertiary_category_id,brand_id,global_product_id
117974,1992189181,105073563,319.0,226.0,226.0,17407.0,811784
128310,1992189181,105073190,319.0,226.0,226.0,17407.0,811784


In [14]:
# Since it looks like an isolated anomoly, probably best to remove from analysis completely
# Note: if used as a SQL primary key, this error would mess things up
ri.drop_duplicates(subset='receipt_item_id', inplace=True, ignore_index=True, keep=False)
rid.drop_duplicates(subset='receipt_item_id', inplace=True, ignore_index=True, keep=False)

# Now the odd duplicated id is gone
rid[rid['receipt_item_id'] == 1992189181]

,receipt_item_id,receipt_id,primary_category_id,secondary_category_id,tertiary_category_id,brand_id,global_product_id


#### Note:
Probably worth looking up the two receipts where this anomoly came from and see if you can figure out why it happened. Both receipts were posted pretty close only a few minutes apart. That is, if that is allowed in terms of privacy and legality.

In [15]:
r[(r['id'] == 105073190) | (r['id'] == 105073563)]

,id,customer_id,retailer_id,created_at,total_price
13859,105073190,10968001,4.0,2016-09-21 03:37:15,286.54
15732,105073563,10968001,4.0,2016-09-21 03:41:12,450.25


### == missing data analysis == 
Though not the only thing missing, price and quantity were the biggest areas of missing data.

In [16]:
# Merging all three "tables" for easier analysis.

# First two
halfdf = rid.merge(r, left_on='receipt_id', right_on='id', validate='many_to_one')

# Adding third
df = halfdf.merge(ri, on='receipt_item_id')

df.drop(columns='id', inplace=True)
df

,receipt_item_id,receipt_id,primary_category_id,secondary_category_id,tertiary_category_id,brand_id,global_product_id,customer_id,retailer_id,created_at,total_price,price,quantity
0,1930961526,101782310,130.0,131.0,131.0,4477.0,525012,12531001,4.0,2016-09-06 12:48:35,28.98,NaN,NaN
1,1930961527,101782310,NaN,NaN,NaN,34236.0,891808,12531001,4.0,2016-09-06 12:48:35,28.98,NaN,NaN
2,1930961529,101782310,237.0,240.0,246.0,34246.0,156367,12531001,4.0,2016-09-06 12:48:35,28.98,NaN,NaN
3,1930961531,101782310,130.0,132.0,132.0,4517.0,71186,12531001,4.0,2016-09-06 12:48:35,28.98,NaN,NaN
4,1930961532,101782310,223.0,369.0,369.0,17407.0,750345,12531001,4.0,2016-09-06 12:48:35,28.98,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
240544,2318337713,123401030,237.0,255.0,262.0,34246.0,751626,3785001,4.0,2016-12-05 21:20:26,32.78,NaN,NaN
240545,2319598027,123464264,58.0,70.0,70.0,677.0,5973,5865001,9.0,2016-12-06 00:31:46,31.60,4.49,NaN
240546,2319598033,123464264,58.0,70.0,70.0,708.0,5991,5865001,9.0,2016-12-06 00:31:46,31.60,4.49,NaN
240547,2319598053,123464264,22.0,23.0,23.0,1019.0,9343,5865001,9.0,2016-12-06 00:31:46,31.60,2.29,NaN


In [17]:
# Finding receipts with no missing prices or quantities

# For each receipt, calculate how many prices or quantities are missing
df3 = df.groupby('receipt_id').agg({'price':lambda x:x.isnull().sum(),'quantity':lambda x:x.isnull().sum()})

# How many receipts are not missing anything?
df3[(df3['price'] == 0) & (df3['quantity'] ==0)].shape

(1153, 2)

So 1,153 out of 22,321 reciepts were not missing any price or quantity data, about 5%. Not very high.

This tells us that there is a lot of information not being captured by the OCR system.

In [18]:
# Trying to reconstruct the total price from info in the receipt

# Take out rows with missing info
df1 = df[~df['price'].isna() & ~df['quantity'].isna()]

# Multiply price by quantity
df1['price_est'] = df1['price'] * df['quantity']

# Group by receipt to make a total price estimate
df2 = df1.groupby('receipt_id').agg({'total_price':np.max, 'price_est':np.sum})

# Some estimates are close (see row 2), while others are not (see rows 1 and 3)
df2

/tmp/ipykernel_12732/3551776.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['price_est'] = df1['price'] * df['quantity']


,total_price,price_est
receipt_id,,
101713484,61.74,4.98
101792228,6.00,6.98
101792589,2.50,757.35
101795183,17.54,10.71
101795781,46.63,38.15
...,...,...
123359214,82.18,3.38
123478605,338.60,25.47
123486130,56.74,1.00


In [19]:
df2[df2['total_price'] == df2['price_est']].shape

(280, 2)

In [20]:
# Quick function to tell us how many rows of our calculated prices were within 20% of the actual total
np.isclose(df2['price_est'], df2['total_price'], atol=0, rtol=0.2).sum()

656

When we take those 5% of receipts with no missing information and try to reconstruct the total price, only 280 receipts matched exactly. Even if you give a 20% price tolerance to allow for possible taxes, discounts, surcharges, etc. you only have 656 receipts that match. That's 3% of the total receipts. Again not very high.

So even among the receipts we consider "complete," we have evidence that at least half are still missing price related information. It could be tax, discounts, or coupons. It could be an OCR error like misreading numbers or missing items on receipts. Either way it is worth investigating some of the receipts that did and did not have price estimate matches to tease out systemic errors.

### == price analysis ==

Let's check on the prices of things.

In [21]:
# How many products don't have a price anywhere?

df5 = df.groupby('global_product_id').agg('count')
df5

,receipt_item_id,receipt_id,primary_category_id,secondary_category_id,tertiary_category_id,brand_id,customer_id,retailer_id,created_at,total_price,price,quantity
global_product_id,,,,,,,,,,,,
5,7,7,7,7,7,7,7,7,7,6,6,1
9,1,1,1,1,1,1,1,1,1,1,0,1
10,1,1,1,1,1,1,1,1,1,1,0,1
12,1,1,1,1,1,1,1,1,1,1,0,1
14,3,3,3,3,3,3,3,3,3,3,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...
960308,1,1,0,0,0,1,1,1,1,1,0,0
960329,2,2,0,0,0,2,2,2,2,2,0,2
960330,2,2,0,0,0,2,2,2,2,2,0,2


In [22]:
# About 45% have no price anywhere
df5[df5['price'] == 0].shape

(24253, 12)

In [23]:
# The highest price is $71 billion for a single item. Not realistic.
df['price'].max()

71290200000.0

In [24]:
# There are 72 items that cost more than $100,000
df[df['price'] > 100000].shape

(72, 13)

Are there different prices for different rows with the same global_product_id?

In [25]:
# Find the min and max price for each product
df4 = df[~df['price'].isna()].groupby('global_product_id').agg({'price':[np.max, np.min]})

# Find the difference
df4['price_diff'] = df4['price', 'amax'] - df4['price', 'amin']

df4

price        price_diff
                    amax   amin           
global_product_id                         
5                   6.58   2.98       3.60
41                  3.38   3.38       0.00
142                 3.79   3.79       0.00
144                 2.73   2.00       0.73
145                 4.46   4.46       0.00
...                  ...    ...        ...
960253              1.98   1.98       0.00
960256              3.99   3.00       0.99
960273              2.59   2.59       0.00
960406             11.97  11.97       0.00
960436              5.95   5.95       0.00

[29121 rows x 3 columns]

In [26]:
# 9,271 of 29,121 products (32%) have more than one price

df4[df4['price_diff'] != 0].shape

(9271, 3)

In [27]:
# More than 1,000 have a difference of more than $10

df4[df4['price_diff'] > 10].shape

(1124, 3)

In [28]:
# More than 100 have a difference of more than $100

df4[df4['price_diff'] > 100].shape

(147, 3)

In [29]:
# Fewer but not an insignificant amount have a difference of more than $100,000

df4[df4['price_diff'] > 100000].shape

(25, 3)

So there are clear price errors. Some prices even reach into the billions, which means either somebody is buying a small country or a price is being misread. 

There are also products that have different prices. This in of itself is not surprising, discounts happen all the time after all. But there are some clear outliers, mostly linked to the incorrect price listings.

These can both be solved by removing errors, but it's difficult to say where the line is. Do we remove anything priced above $10,000? $1,000? $500? There may not be a correct answer and you have to make a judgment call that balances accuracy and inevitably lost data.